In [4]:
from symmer.operators import PauliwordOp
from symmer.projection import ContextualSubspace
from symmer import QuantumState
from symmer.utils import stab_renyi_entropy
import sys
from scipy import sparse
from itertools import product, chain

sys.path.append('../')

import numpy as np
from IPython.display import HTML
import time
from qiskit._accelerate.sparse_pauli_op import (
    ZXPaulis,
    to_matrix_sparse,
)

In [2]:
random_state=QuantumState.haar_random(n_qubits=8)
t1=time.time()
m2_true=stab_renyi_entropy(random_state,filtered=False)
t2=time.time()
print(m2_true)
print(t2-t1)

6.01598814722005
11.838296890258789


In [3]:
t1=time.time()
m2_sample=stab_renyi_entropy(random_state,sampling=True,n_samples=1e6,filtered=False,sampling_approach='Metropolis')
t2=time.time()
print(m2_sample)
print(t2-t1)

6.710629778486246
253.4937448501587


In [20]:
this_op=PauliwordOp.from_dictionary({'ZZZZ':1})
this_sparse=this_op.to_sparse_matrix

In [28]:
rng= np.random.default_rng()


In [29]:
t1=time.time()
for i in range (10):
    iter_vec=np.zeros(2*4,dtype=bool)
    [int1,int2]=rng.integers(low=1,high=2*4,size=2)
    iter_vec[int1]=1
    iter_vec[int2]=1
    iter_op=PauliwordOp(symp_matrix=iter_vec,coeff_vec=[1])
t2=time.time()
print(t2-t1)

0.0004699230194091797


In [ ]:
n_qubits=4
t1=time.time()
#paulis = (X, Z, Y)
paulis=(np.array([[0,1],[1,0]]),np.array([[1,0],[0,-1]]),np.array([[0,-1],[1,0]]))
iter_sparse=sparse.identity(2**n_qubits)
for i in range(10):
    hops=rng.choice(np.arange(2*n_qubits),size=2).tolist()
    for rand_int in hops:
        which_q=int(rand_int/2)
        which_op=rand_int%2
        if which_q==0:
            #first qubit
            iter_sparse.dot(sparse.block_diag([paulis[which_op],np.identity(2**(n_qubits-1))]))
        elif which_q==n_qubits:
            iter_sparse.dot(sparse.block_diag([np.identity(2**(n_qubits-1)),paulis[which_op]]))
        else:
            iter_sparse.dot(sparse.block_diag([np.identity(2**which_q),paulis[which_op],np.identity(2**(n_qubits-which_q-1))]))
t2=time.time()
print(t2-t1)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [22]:
%timeit iter_op=PauliwordOp(symp_matrix=iter_vec,coeff_vec=[1]); next_op=this_op * iter_op; next_op.to_sparse_matrix

118 μs ± 230 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
%timeit iter_sparse=PauliwordOp(symp_matrix=iter_vec,coeff_vec=[1]).to_sparse_matrix; this_sparse.dot(iter_sparse)

112 μs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
%timeit sparse_hop=sparse.block_diag((np.identity(2**2),np.array([[1,0],[0,-1]]),np.identity(2)))

29 μs ± 582 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
[int1,int2]=rng.choice(np.arange(2*4),size=2).tolist()


5

In [47]:
np.identity(2)

array([[1., 0.],
       [0., 1.]])

In [59]:
symp_vec=list(map(bool,symp_list[15]))

%timeit phase = np.zeros(1, dtype=np.uint8) ;zx = ZXPaulis(np.array([symp_vec[:10]])[:,::-1],np.array([symp_vec[10:]])[:,::-1],phase,np.array([1+0j]),);data, indices, indptr = to_matrix_sparse(zx, force_serial=False);side = 1 << 10;pauli_mat= sparse.csr_matrix((data, indices, indptr), shape=(side, side))

111 μs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
